In [244]:
from symred.S3_projection import S3_projection, unitary_partitioning_rotations
from symred.symplectic_form import *
from scipy.optimize import shgo, differential_evolution
from symred.utils import gf2_gaus_elim, unit_n_sphere_cartesian_coords
from functools import reduce
import json

class CS_VQE(S3_projection):
    """
    """
    def __init__(self,
            operator: PauliwordOp,
            ref_state: np.array = None,
            target_sqp: str = 'Z',
            basis_weighting_operator: PauliwordOp = None
        ) -> None:
        """ 
        """
        self.operator = operator
        self.ref_state = ref_state
        self.target_sqp = target_sqp
        if basis_weighting_operator is not None:
            self.basis_weighting_operator = basis_weighting_operator
        else:
            self.basis_weighting_operator = operator
        self.contextual_operator = (operator-self.noncontextual_operator).cleanup_zeros()
        # decompose the noncontextual set into a dictionary of its 
        # universally commuting elements and anticommuting cliques
        self.noncontextual_reconstruction = (
            self.noncontextual_operator.basis_reconstruction(self.noncontextual_basis)
        )
        self.r_indices = self.noncontextual_reconstruction[:,:self.n_cliques]
        self.G_indices = self.noncontextual_reconstruction[:,self.n_cliques:]
        self.clique_operator = (self.noncontextual_basis[:self.n_cliques]).sort(key='Z')
        symmetry_generators_symp = self.noncontextual_basis.symp_matrix[self.n_cliques:]
        self.symmetry_generators = StabilizerOp(
            symmetry_generators_symp,
            np.ones(symmetry_generators_symp.shape[0])
        )
        # determine the noncontextual ground state - this updates the coefficients of the clique 
        # representative operator C(r) and symmetry generators G with the optimal configuration
        self.solve_noncontextual(ref_state)
        
    def basis_score(self, 
            basis: StabilizerOp
        ) -> float:
        """ Evaluate the score of an input basis according 
        to the basis weighting operator, for example:
            - set Hamiltonian cofficients to 1 for unweighted number of commuting terms
            - specify as the SOR Hamiltonian to weight according to second-order response
            - input UCC operator to weight according to coupled-cluster theory <- best performance
            - if None given then weights by Hamiltonian coefficient magnitude
        """
        # mask terms of the weighting operator that are preserved under projection over the basis
        mask_preserved = np.where(np.all(self.basis_weighting_operator.commutes_termwise(basis),axis=1))[0]
        return (
            np.linalg.norm(self.basis_weighting_operator.coeff_vec[mask_preserved]) /
            np.linalg.norm(self.basis_weighting_operator.coeff_vec)
            )
    
    def update_eigenvalue(self, stabilizer: StabilizerOp) -> None:
        """ Update the +/-1 eigenvalue assigned to the input stabilizer
        according to the noncontextual ground state configuration
        """
        stabilizer.coeff_vec[0] = (-1) ** np.count_nonzero(
            np.bitwise_and(
                stabilizer.basis_reconstruction(self.symmetry_generators)==1, 
                self.symmetry_generators.coeff_vec==-1
                )
            )

    @cached_property
    def noncontextual_operator(self) -> PauliwordOp:
        """ Extract a noncontextual set of Pauli terms from the operator

        Implementation of the algorithm in https://doi.org/10.1103/PhysRevLett.123.200501
        Does a single pass over the Hamiltonian and appends terms to noncontextual_operator 
        that do not make it contextual - easy to do multiple passes although this does not 
        seem to yield better results from experimentation.

        TODO graph-based approach, currently uses legacy implementation
        """
        # order the operator terms by coefficient magnitude
        check_ops = self.operator.sort(key='magnitude')
        # initialise as identity with 0 coefficient
        I_symp = np.zeros(2*self.operator.n_qubits, dtype=int)
        noncontextual_operator = PauliwordOp(I_symp, [0])
        for i in range(check_ops.n_terms):
            if (noncontextual_operator+check_ops[i]).is_noncontextual:
                noncontextual_operator+=check_ops[i]
        return noncontextual_operator

    @cached_property
    def noncontextual_basis(self) -> StabilizerOp:
        """ Find an independent basis for the noncontextual symmetry
        """
        self.decomposed = {}
        # extract the universally commuting noncontextual terms
        universal_mask = np.where(np.all(self.noncontextual_operator.adjacency_matrix, axis=1))
        universal_operator = PauliwordOp(self.noncontextual_operator.symp_matrix[universal_mask],
                                         self.noncontextual_operator.coeff_vec[universal_mask])
        self.decomposed['symmetry'] = universal_operator
        # identify the anticommuting cliques
        clique_union = (self.noncontextual_operator - universal_operator).cleanup_zeros()
        # order lexicographically and take difference between adjacent rows
        clique_grouping_order = np.lexsort(clique_union.adjacency_matrix.T)
        diff_adjacent = np.diff(clique_union.adjacency_matrix[clique_grouping_order], axis=0)
        # the unique cliques are the non-zero rows in diff_adjacent
        mask_unique_cliques = np.append(True, ~np.all(diff_adjacent==0, axis=1))
        # determine the inverse mapping so terms of the same clique have the same index
        inverse_index = np.zeros_like(clique_grouping_order)
        inverse_index[clique_grouping_order] = np.cumsum(mask_unique_cliques) - 1
        mask_cliques = np.stack([np.where(inverse_index==i)[0] for i in np.unique(inverse_index)])
        # mask each clique and select a class represetative for its contribution in the noncontextual basis
        clique_reps = []
        for i, (Ci_symp, Ci_coef) in enumerate(
            zip(
                clique_union.symp_matrix[mask_cliques],
                clique_union.coeff_vec[mask_cliques]
            )
        ):
            Ci_operator = PauliwordOp(Ci_symp, Ci_coef)
            self.decomposed[f'clique_{i}'] = Ci_operator
            # choose cliques representative that maximises basis_score
            rep_scores = [(Ci_operator[i], self.basis_score(Ci_operator[i])) for i in range(len(Ci_coef))]
            clique_reps.append(sorted(rep_scores, key=lambda x:-x[1])[0][0].symp_matrix)

        # now we are ready to build the noncontextual basis...
        # perform Gaussian elimination on the symmetry terms:
        reduced_universal = gf2_gaus_elim(universal_operator.symp_matrix)
        reduced_universal = reduced_universal[np.where(np.any(reduced_universal, axis=1))]
        basis = PauliwordOp(reduced_universal, np.ones(reduced_universal.shape[0]))
        clique_reps = np.vstack(clique_reps)
        basis = basis + PauliwordOp(clique_reps, np.ones(clique_reps.shape[0]))
        basis_order = np.lexsort(basis.adjacency_matrix)
        basis = StabilizerOp(basis.symp_matrix[basis_order],np.ones(basis.n_terms))
        self.n_cliques = np.count_nonzero(~np.all(basis.adjacency_matrix, axis=1))
        
        return basis
    
    def noncontextual_objective_function(self, 
            nu: np.array, 
            r: np.array
        ) -> float:
        """ The classical objective function that encodes the noncontextual energies
        """
        G_prod = (-1)**np.count_nonzero(np.logical_and(self.G_indices==1, nu == -1), axis=1)
        r_part = np.sum(self.r_indices*r, axis=1)
        r_part[np.where(r_part==0)]=1
        return np.sum(self.noncontextual_operator.coeff_vec*G_prod*r_part).real

    def solve_noncontextual(self, ref_state: np.array = None) -> None:
        """ Minimize the classical objective function, yielding the noncontextual ground state
        """
        def convex_problem(nu):
            """ given +/-1 value assignments nu, solve for the clique operator coefficients.
            Note that, with nu fixed, the optimization problem is now convex.
            """
            # given M cliques, optimize over the unit (M-1)-sphere and convert to cartesians for the r vector
            r_bounds = [(0, np.pi)]*(self.n_cliques-2)+[(0, 2*np.pi)]
            optimizer_output = differential_evolution(
                func=lambda angles:self.noncontextual_objective_function(
                    nu, unit_n_sphere_cartesian_coords(angles)
                    ), 
                bounds=r_bounds
            )
            optimized_energy = optimizer_output['fun']
            optimized_angles = optimizer_output['x']
            r_optimal = unit_n_sphere_cartesian_coords(optimized_angles)
            return optimized_energy, r_optimal

        if ref_state is None:
            # optimize discrete value assignments nu by relaxation to continuous variables
            nu_bounds = [(0, np.pi)]*self.symmetry_generators.n_terms
            optimizer_output = shgo(func=lambda angles:convex_problem(np.cos(angles))[0], bounds=nu_bounds)
            # if optimization was successful the optimal angles should consist of 0 and pi
            self.symmetry_generators.coeff_vec = np.array(np.cos(optimizer_output['x']), dtype=int)
        else:
            # update the symmetry generator G coefficients w.r.t. the reference state
            self.symmetry_generators.update_sector(ref_state=ref_state)
        
        # optimize the clique operator coefficients
        fix_nu = self.symmetry_generators.coeff_vec
        self.noncontextual_energy, r = convex_problem(fix_nu)
        self.clique_operator.coeff_vec = r
        
    def contextual_subspace_projection(self,
            stabilizers: List[PauliwordOp],
            aux_operator: PauliwordOp = None
        ) -> PauliwordOp:
        """ input a list of independent operators one wishes to map onto single-qubit 
        Pauli operators and project into the corresponding stabilizer subspace
        """
        # define the operator to be projected (aux_operator faciliates ansatze to be projected)
        if aux_operator is not None:
            operator_to_project = aux_operator.copy()
        else:
            operator_to_project = self.operator.copy()
        
        insert_rotations = []
        fix_stabilizers = []
        for stab in stabilizers:
            if stab.n_terms > 1:
                # if any stabilizers in the list contain more than one term then apply unitary partitioning
                UP_rot = unitary_partitioning_rotations(stab)
                insert_rotations+=UP_rot
                stab = stab.recursive_rotate_by_Pword(UP_rot).cleanup_zeros()
            else:
                self.update_eigenvalue(stab)
            fix_stabilizers.append(stab)
        fix_stabilizers = reduce(lambda x,y:x+y, fix_stabilizers)
                
        # instantiate as StabilizerOp to ensure algebraic independence and coefficients are +/-1
        fix_stabilizers = StabilizerOp(
            fix_stabilizers.symp_matrix, 
            np.array(fix_stabilizers.coeff_vec, dtype=int),
            target_sqp=self.target_sqp
        )
        # instantiate the parent S3_projection classwith the stabilizers we are enforcing
        super().__init__(fix_stabilizers, target_sqp=self.target_sqp)

        return self.perform_projection(
            operator=operator_to_project,
            insert_rotations=insert_rotations
        )

In [245]:
with open('data/molecule_data.json', 'r') as jfile:
    molecule_geometries = json.load(jfile)
print(molecule_geometries.keys())

dict_keys(['H2_3-21G_SINGLET', 'H6_STO-3G_SINGLET', 'H2_6-31G_SINGLET', 'H2_6-311G_SINGLET', 'H3+_STO-3G_SINGLET', 'H3+_3-21G_SINGLET', 'HeH+_3-21G_SINGLET', 'HeH+_6-311G_SINGLET', 'H2O_STO-3G_SINGLET', 'BeH+_STO-3G_SINGLET', 'LiH_STO-3G_SINGLET', 'CH+_STO-3G_SINGLET', 'HF_STO-3G_SINGLET', 'B+_STO-3G_SINGLET', 'B_STO-3G_DOUBLET', 'N_STO-3G_QUARTET', 'OH-_STO-3G_SINGLET', 'O_STO-3G_TRIPLET', 'CH2_STO-3G_TRIPLET', 'BeH2_STO-3G_SINGLET', 'Be_STO-3G_SINGLET', 'C_STO-3G_TRIPLET', 'NH_STO-3G_SINGLET', 'Ne_STO-3G_SINGLET', 'F_STO-3G_DOUBLET', 'Li_STO-3G_DOUBLET', 'BH_STO-3G_SINGLET', 'NeH+_STO-3G_SINGLET', 'NH2-_STO-3G_SINGLET', 'BH3_STO-3G_SINGLET', 'BH2+_STO-3G_SINGLET', 'HCl_STO-3G_SINGLET', 'H4_STO-3G_SINGLET', 'NH3_STO-3G_SINGLET', 'F2_STO-3G_SINGLET', 'HCN_STO-3G_SINGLET', 'CH4_STO-3G_SINGLET', 'CH3NHCH3_STO-3G_SINGLET', 'CH3CH2NH2_STO-3G_SINGLET', 'CH3CH2OH_STO-3G_SINGLET', 'CH3OH_STO-3G_SINGLET', 'C2H6_STO-3G_SINGLET', 'CH3CN_STO-3G_SINGLET', 'CH3CHO_STO-3G_SINGLET', 'CH3CHOHCH3_STO-3

In [264]:
# Set molecule parameters
speciesname = 'Be_STO-3G_SINGLET'
mol_data = molecule_geometries[speciesname]
if 'name' in mol_data:
    print(mol_data['name'])
    
atoms = mol_data['atoms']
coords = mol_data['coords']
basis = mol_data['basis']
multiplicity = mol_data['multiplicity']
charge = mol_data['charge']
geometry = list(zip(atoms, coords))

xyz_file = str(len(atoms))+'\n '

for atom, coords in geometry:
    xyz_file += '\n'+atom+'\t'
    xyz_file += '\t'.join(list(map(str, coords)))

print('Molecule geometry:')
print(xyz_file[4:])

Molecule geometry:
Be	0	0	0


In [265]:
from symred.chem import PySCFDriver

convergence = 1e6; max_hf_cycles=100_000; ram = 8_000
run_mp2  = True; run_cisd = False; run_ccsd = True; run_fci  = True

pyscf_obj = PySCFDriver(xyz_file,
                        basis,
                        #convergence=convergence,
                        charge=charge,
                        #max_ram_memory=ram,
                        #max_hf_cycles=max_hf_cycles,                   
                        run_mp2=run_mp2,
                        run_cisd=run_cisd,
                        run_ccsd=run_ccsd,
                        run_fci=run_fci)

pyscf_obj.run_pyscf()

print('HF converged?  ', pyscf_obj.pyscf_hf.converged)
print('CCSD converged?', pyscf_obj.pyscf_ccsd.converged)
print('FCI converged? ', pyscf_obj.pyscf_fci.converged)

hf_energy = pyscf_obj.pyscf_hf.e_tot
mp2_energy = pyscf_obj.pyscf_mp2.e_tot
ccsd_energy = pyscf_obj.pyscf_ccsd.e_tot
fci_energy = pyscf_obj.pyscf_fci.e_tot

print()
print(f'HF energy: {hf_energy}')
print(f'MP2 energy: {mp2_energy}')
print(f'CCSD energy: {ccsd_energy}')
print(f'FCI energy: {fci_energy}')

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy: -14.351880476202023
MP2 energy: -14.376238850841515
CCSD energy: -14.403650683524738
FCI energy: -14.403655108067682


In [266]:
from symred.chem import FermionicHamilt, FermioniCC
from openfermion import get_fermion_operator, jordan_wigner, hermitian_conjugated

H_fermion = FermionicHamilt(pyscf_obj.pyscf_hf)
T_fermion = FermioniCC(pyscf_obj.pyscf_ccsd)
H_fermion.build_operator()
T_fermion.build_operator()

n_qubits = H_fermion.n_qubits
print('Number of qubits:', n_qubits)

H = get_fermion_operator(H_fermion.fermionic_molecular_hamiltonian)
T = T_fermion.fermionic_cc_operator
T -= hermitian_conjugated(T)

H_jw = jordan_wigner(H)
T_jw = jordan_wigner(T)

Number of qubits: 10


In [267]:
from openfermion import QubitOperator

def QubitOperator_to_dict(op, num_qubits):
    assert(type(op) == QubitOperator)
    op_dict = {}
    term_dict = op.terms
    terms = list(term_dict.keys())

    for t in terms:    
        letters = ['I' for i in range(num_qubits)]
        for i in t:
            letters[i[0]] = i[1]
        p_string = ''.join(letters)        
        op_dict[p_string] = term_dict[t]
         
    return op_dict

H_q = PauliwordOp(QubitOperator_to_dict(H_jw, n_qubits))
T_q = PauliwordOp(QubitOperator_to_dict(T_jw, n_qubits))
T_q.coeff_vec = T_q.coeff_vec.imag

In [268]:
print(T_q)

-0.0001058066 YYIIYXIIII +
0.0001058066 YXIIYYIIII +
0.0001058066 XXIIYXIIII +
0.0001058066 XYIIYYIIII +
-0.0001058066 YXIIXXIIII +
-0.0001058066 YYIIXYIIII +
-0.0001058066 XYIIXXIIII +
0.0001058066 XXIIXYIIII +
-0.0004444204 YZZYYXIIII +
0.0004444204 YZZXYYIIII +
0.0004444204 XZZXYXIIII +
0.0004444204 XZZYYYIIII +
-0.0004444204 YZZXXXIIII +
-0.0004444204 YZZYXYIIII +
-0.0004444204 XZZYXXIIII +
0.0004444204 XZZXXYIIII +
0.0004444204 IYYIYXIIII +
-0.0004444204 IYXIYYIIII +
-0.0004444204 IXXIYXIIII +
-0.0004444204 IXYIYYIIII +
0.0004444204 IYXIXXIIII +
0.0004444204 IYYIXYIIII +
0.0004444204 IXYIXXIIII +
-0.0004444204 IXXIXYIIII +
-0.0246191655 IIYYYXIIII +
0.0246191655 IIYXYYIIII +
0.0246191655 IIXXYXIIII +
0.0246191655 IIXYYYIIII +
-0.0246191655 IIYXXXIIII +
-0.0246191655 IIYYXYIIII +
-0.0246191655 IIXYXXIIII +
0.0246191655 IIXXXYIIII +
-0.0001058066 YYIIIIYXII +
0.0001058066 YXIIIIYYII +
0.0001058066 XXIIIIYXII +
0.0001058066 XYIIIIYYII +
-0.0001058066 YXIIIIXXII +
-0.0001058066 YYIIII

In [269]:
from symred.S3_projection import QubitTapering

taper_hamiltonian = QubitTapering(H_q)

print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
print('The symmetry generators are\n')
print(taper_hamiltonian.symmetry_generators)
print('\nand are rotated onto the single-qubit Pauli operators\n')
print(taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis())
print('\nvia a sequence of Clifford pi/2 rotations\n')
print(taper_hamiltonian.stabilizers.stabilizer_rotations)

We are able to taper 5 qubits from the Hamiltonian.

The symmetry generators are

1+0j ZIZIIZIZIZ +
1+0j IZIZIZIZIZ +
1+0j IIIIZZIIII +
1+0j IIIIIIZZII +
1+0j IIIIIIIIZZ

and are rotated onto the single-qubit Pauli operators

-1+0j IIIIIIIIXI +
-1+0j IIIIIIXIII +
-1+0j IIIIXIIIII +
-1+0j IXIIIIIIII +
-1+0j XIIIIIIIII

via a sequence of Clifford pi/2 rotations

[('IIIIYZIIII', None), ('IIIIIIIIYZ', None), ('IIIIIIYZII', None), ('IYIZIZIZIZ', None), ('YIZIIZIZIZ', None)]


In [270]:
hf_array = H_fermion.hf_comp_basis_state
taper_hamiltonian.stabilizers.update_sector(hf_array)
print(f'The symmetry sector corresponding with the single reference {hf_array} is {taper_hamiltonian.stabilizers.coeff_vec}')

The symmetry sector corresponding with the single reference [1 1 1 1 0 0 0 0 0 0] is [1 1 1 1 1]


In [271]:
ham_tap = taper_hamiltonian.taper_it(ref_state=hf_array)
#sor_tap = taper_hamiltonian.taper_it(aux_operator=sor_ham, ref_state=hf_array)
#sor_tap.coeff_vec/=np.linalg.norm(sor_tap.coeff_vec)
ucc_tap = taper_hamiltonian.taper_it(aux_operator=T_q, ref_state=hf_array)
n_taper = taper_hamiltonian.n_taper
tapered_qubits   = taper_hamiltonian.stab_qubit_indices
untapered_qubits = taper_hamiltonian.free_qubit_indices
hf_tapered = taper_hamiltonian.tapered_ref_state

dashes = "------------------------------------------------"
print("Tapering information:")
print(dashes)
print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian')
print('The symmetry basis/sector is:') 
print(taper_hamiltonian.symmetry_generators)
print(f'The tapered Hartree-Fock state is', QuantumState([hf_tapered]))
print(dashes)

Tapering information:
------------------------------------------------
We are able to taper 5 qubits from the Hamiltonian
The symmetry basis/sector is:
1 ZIZIIZIZIZ +
1 IZIZIZIZIZ +
1 IIIIZZIIII +
1 IIIIIIZZII +
1 IIIIIIIIZZ
The tapered Hartree-Fock state is  1.0000000000 |11000>
------------------------------------------------


In [272]:
cs_vqe = CS_VQE(ham_tap, hf_tapered, basis_weighting_operator=ucc_tap)

In [273]:
print(cs_vqe.noncontextual_energy)

-14.351880476201988


In [274]:
hf_energy

-14.351880476202023

In [287]:
print(cs_vqe.decomposed['symmetry'])

-8.6720073786+0.0000000000j IIIII +
-0.2968649525+0.0000000000j IIIIZ +
-0.2968649525+0.0000000000j IIIZI +
0.3892355930+0.0000000000j IIIZZ +
-0.2968649525+0.0000000000j IIZII +
0.3892355930+0.0000000000j IIZIZ +
0.3892355930+0.0000000000j IIZZI +
0.2634692003+0.0000000000j IIZZZ +
-0.0466916923+0.0000000000j IZIII +
0.1830962071+0.0000000000j IZIIZ +
0.1830962071+0.0000000000j IZIZI +
0.2807146486+0.0000000000j IZIZZ +
0.1830962071+0.0000000000j IZZII +
0.2807146486+0.0000000000j IZZIZ +
0.2807146486+0.0000000000j IZZZI +
2.2432292443-0.0000000000j IZZZZ


In [289]:
print(cs_vqe.symmetry_generators)
print()
print(cs_vqe.clique_operator)

1 IIIIZ +
1 IIIZI +
1 IIZII +
-1 IZIII

-1.0000000000 ZIZZZ +
-0.0000001784 XIIII


In [290]:
UP_rot = unitary_partitioning_rotations(cs_vqe.clique_operator)

In [291]:
stab = cs_vqe.clique_operator.recursive_rotate_by_Pword(UP_rot).cleanup_zeros()
print(stab)
cs_vqe.update_eigenvalue(stab)
print(stab)

-1.0000000000+0.0000000000j ZIZZZ
1.0000000000+0.0000000000j ZIZZZ


In [307]:
S = PauliwordOp({'IZZZZ':0})
print(S)
cs_vqe.update_eigenvalue(S)
print(S)

0.0000000000+0.0000000000j IZZZZ
-1.0000000000+0.0000000000j IZZZZ


In [314]:
stabs = [cs_vqe.clique_operator, PauliwordOp({'IZZZZ':10000})]
ham_cs = cs_vqe.contextual_subspace_projection(stabs)

/tmp/ipykernel_24488/1852410258.py:216: ComplexWarning: Casting complex values to real discards the imaginary part
  np.array(fix_stabilizers.coeff_vec, dtype=int),


In [315]:
from symred.utils import exact_gs_energy

exact_gs_energy(ham_cs.to_sparse_matrix)[0] - fci_energy

0.00032650444009618695

In [316]:
cs_vqe.basis_weighting_operator = ucc_tap #cs_vqe.basis_weighting_operator.recursive_rotate_by_Pword(UP_rot)
cs_vqe.basis_score(reduce(lambda x,y:x+y, [cs_vqe.clique_operator.recursive_rotate_by_Pword(UP_rot).cleanup_zeros()]))

0.9998279314217895

In [331]:
random_stabs = []
n_stabs = 3
for i in range(10000):
    random_Z_block = np.random.randint(0,2,(n_stabs,ham_tap.n_qubits))
    symp_matrix = np.hstack([np.zeros_like(random_Z_block), random_Z_block])
    try:
        basis = StabilizerOp(symp_matrix, np.ones(n_stabs))
        random_stabs.append([cs_vqe.basis_score(basis), basis])
    except:
        pass
    
score, stabs = sorted(random_stabs,key = lambda x:-x[0])[0]
stabs = [stabs[i] for i in range(stabs.n_terms)]
ham_cs = cs_vqe.contextual_subspace_projection(stabs)

print(score)
print(stabs)
exact_gs_energy(ham_cs.to_sparse_matrix)[0] - fci_energy

0.8162231079921426
[<symred.symplectic_form.PauliwordOp object at 0x7fb3bf4051f0>, <symred.symplectic_form.PauliwordOp object at 0x7fb3bf552760>, <symred.symplectic_form.PauliwordOp object at 0x7fb3bf36d580>]


/tmp/ipykernel_24488/1852410258.py:216: ComplexWarning: Casting complex values to real discards the imaginary part
  np.array(fix_stabilizers.coeff_vec, dtype=int),


4.373578894064025

In [201]:
ham_tap

In [104]:
print(cs_vqe.basis_weighting_operator)

-0.0002116131+0.0000000000j IIIIY +
-0.0000000001+0.0000000000j IIZZY +
0.0000000001+0.0000000000j IZIIY +
0.0002116131+0.0000000000j IZZZY +
0.0002116131+0.0000000000j ZIZZY +
-0.0002116131+0.0000000000j ZZIIY +
-0.0002116131+0.0000000000j IIIYI +
-0.0000000001+0.0000000000j IIZYZ +
0.0000000001+0.0000000000j IZIYI +
0.0002116131+0.0000000000j IZZYZ +
0.0002116131+0.0000000000j ZIZYZ +
-0.0002116131+0.0000000000j ZZIYI +
-0.0002116131+0.0000000000j IIYII +
-0.0000000001+0.0000000000j IIYZZ +
0.0000000001+0.0000000000j IZYII +
0.0002116131+0.0000000000j IZYZZ +
0.0002116131+0.0000000000j ZIYZZ +
-0.0002116131+0.0000000000j ZZYII +
0.0008888408+0.0000000000j IXIIY +
-0.0000000037+0.0000000000j IXZZY +
-0.0008888408+0.0000000000j IYIIX +
0.0000000037+0.0000000000j IYZZX +
-0.0008888408+0.0000000000j ZXZZY +
0.0008888408+0.0000000000j ZYZZX +
0.0008888408+0.0000000000j IXIYI +
-0.0000000037+0.0000000000j IXZYZ +
-0.0008888408+0.0000000000j IYIXI +
0.0000000037+0.0000000000j IYZXZ +
-0.000